# CNN MNIST Digit Classification 🔢

In this notebook we will implement an accurate **Handwritten Digit Classification** model. This model will be built using Tensorflow's Keras API.

In [ ]:
# Importing useful modules

import numpy as np
import pandas as pd

import tensorflow as tf

import os


# Printing out the available data
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Reading in the training data into 2 lists: X_train containg the images and Y the labels

X_train = []
Y_train = []

with open('/kaggle/input/digit-recognizer/train.csv') as train_file:
    for line in train_file.readlines()[1:]:
        content = line.strip().split(',')
        label, image_raw_data = content[0], content[1:]
        image_np = np.array_split(np.array(image_raw_data), 28)
        X_train.append(image_np)
        Y_train.append(label)

X_train = np.array(X_train).astype(float)
X_train = np.expand_dims(X_train, axis=3)
Y_train = np.array(Y_train).astype(float)

print(X_train.shape)
print(Y_train.shape)

In [ ]:
# Reading in the testing data into 1 list: X_test containg the images

X_test = []

with open('/kaggle/input/digit-recognizer/test.csv') as test_file:
    for line in test_file.readlines()[1:]:
        content = line.strip().split(',')
        image_raw_data = content[:]
        image_np = np.array_split(np.array(image_raw_data), 28)
        X_test.append(image_np)

X_test = np.array(X_test).astype(float)
X_test = np.expand_dims(X_test, axis=3)

print(X_test.shape)

In [ ]:
# Downloading fonts from Github


!wget --recursive --no-parent 'https://github.com/google/fonts/raw/master/apache/opensans/OpenSans-Regular.ttf' -P /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf
!wget --recursive --no-parent 'https://github.com/google/fonts/raw/master/apache/opensans/OpenSans-Light.ttf' -P /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf
!wget --recursive --no-parent 'https://github.com/google/fonts/raw/master/apache/opensans/OpenSans-SemiBold.ttf' -P /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf
!wget --recursive --no-parent 'https://github.com/google/fonts/raw/master/apache/opensans/OpenSans-Bold.ttf' -P /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf

In [ ]:
# Plotting the output labels distribution

import matplotlib.pyplot as plt
import matplotlib as mpl


mpl.font_manager._rebuild()

plt.rc('font', family='Open Sans')

fig, ax = plt.subplots(figsize=(10, 5), dpi=240)

bars = ax.bar(
    x=np.unique(Y_train, return_counts=True)[0],
    height=np.unique(Y_train, return_counts=True)[1],
    tick_label=np.unique(Y_train, return_counts=True)[0].astype(int)
)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

bar_color = bars[0].get_facecolor()
for bar in bars:
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 50,
        round(bar.get_height(), 1),
        horizontalalignment='center',
        color=bar_color,
        weight='bold'
    )

ax.set_xlabel('Label', labelpad=20, color='#333333', fontsize=12)
ax.set_ylabel('Number of Training Samples', labelpad=16, color='#333333', fontsize=12)
ax.set_title('Label Counts Distribution', pad=14, color='#333333', fontsize=18, weight='bold')

xl, yl, xh, yh=np.array(ax.get_position()).ravel()
w=xh-xl
h=yh-yl
size=0.075

for i in range(10):
    xp=xl+w*(0.06 + (0.106 * i))
    ax1=fig.add_axes([xp-size*0.5, 0.075, size, size])
    ax1.axison = False
    imgplot = ax1.imshow(X_train[np.where(Y_train==i)[0][0]].reshape((28,28)),cmap='Blues', vmin=0, vmax=255)

fig.tight_layout()

In [ ]:
# Plotting 10 examples from each output class

from collections import defaultdict

f, axes = plt.subplots(10, 10, figsize=(16, 18))

counter = defaultdict(int)

for i in range(1000):
    label = int(Y_train[i])
    if counter[label] >= 10: continue
    img = X_train[i].reshape((28,28))
    axes[counter[label], label].imshow(img)
    axes[counter[label], label].set_title(label)
    counter[label] += 1
        
[ax.set_axis_off() for ax in axes.ravel()]
plt.show()

In [ ]:
# Defining the model, compiling and fitting it to the data

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_uniform', padding="SAME"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_initializer = 'he_uniform'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer = 'he_uniform'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer = 'he_uniform'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer = 'he_uniform'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode='max', patience=25)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('./best_model.hdf5', monitor='accuracy',
                                                               mode='max', verbose=1, save_best_model=True)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', patience=15,
                                                          verbose=1, factor=0.5, min_lr=0.0001)

history = model.fit(
    X_train / 255,
    tf.keras.utils.to_categorical(Y_train, num_classes=10),
    epochs=1000,
    shuffle=True,
    callbacks=[early_stopping_callback, model_checkpoint_callback,reduce_lr_callback]
)

In [ ]:
# Plotting the training and pseudo-validation accuracies

acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training Loss')
plt.legend()

plt.show()

In [ ]:
# Predicting on the test set and saving it into a CSV submission file

predictions = model.predict(X_test / 255)
y_pred = np.argmax(predictions, axis=-1)

submission_df = pd.DataFrame(data={
    'ImageId': np.arange(1, X_test.shape[0] + 1),
    'label': y_pred
})
submission_df.to_csv('submission.csv', index=False)
submission_df

In [ ]:
# Plotting 100 prediction examples

f, axes = plt.subplots(10, 10, figsize=(16, 18))

counter = defaultdict(int)

for i in range(100):
    label = int(y_pred[i])
    img = X_test[i].reshape((28,28))
    axes[i % 10, i // 10].imshow(img)
    axes[i % 10, i // 10].set_title(label)
    counter[label] += 1
        
[ax.set_axis_off() for ax in axes.ravel()]
plt.show()